In [1]:
import pandas as pd
import geopandas as gpd
import os
os.getcwd()

'/home/jovyan/PNACaseStudy'

In [2]:
csv_directory = '/home/jovyan/PNACaseStudy'

## Wrangle 2020 effort data

In [14]:
effort2020 = pd.read_csv('combined2020.csv')

In [15]:
grouped_df = effort2020.groupby(['mmsi', 'date', 'cell_ll_lat', 'cell_ll_lon']).agg({
    'hours': 'sum',
    'fishing_hours': 'sum'
}).reset_index()

In [16]:
print(grouped_df.head())

   mmsi        date  cell_ll_lat  cell_ll_lon   hours  fishing_hours
0   156  2020-01-04          6.8         79.4  0.5330         0.2777
1   156  2020-01-04          6.9         79.5  1.6666         0.0000
2   156  2020-01-04          7.0         79.5  0.2630         0.0000
3   156  2020-01-04          7.0         79.6  1.6666         1.6666
4   156  2020-01-04          7.1         79.6  0.2666         0.2666


In [17]:
GFW2020Points = gpd.GeoDataFrame(
    grouped_df, geometry=gpd.points_from_xy(grouped_df.cell_ll_lon, grouped_df.cell_ll_lat), crs="EPSG:4326") 

In [18]:
del(effort2020)

In [19]:
del(grouped_df)

### Clip 2020 GFW data to PNA EEZ boundaries

In [19]:
PNA = gpd.read_file('PNA_EEZs.shp')

In [20]:
print(PNA.head())

     MRGID                                            GEONAME  MRGID_TER1  \
0   8449.0                    Tokelau Exclusive Economic Zone      8675.0   
1   8450.0  Kiribati Exclusive Economic Zone (Phoenix Isla...      8658.0   
2  21798.0  Protected Zone established under the Torres St...      2237.0   
3   8314.0            Solomon Islands Exclusive Economic Zone      8593.0   
4   8318.0           Marshall Islands Exclusive Economic Zone      2226.0   

       POL_TYPE  MRGID_SOV1        TERRITORY1 ISO_TER1        SOVEREIGN1  \
0         200NM      2227.0           Tokelau      TKL       New Zealand   
1         200NM      2116.0     Phoenix Group      KIR          Kiribati   
2  Joint regime      2237.0  Papua New Guinea      PNG  Papua New Guinea   
3         200NM      8593.0   Solomon Islands      SLB   Solomon Islands   
4         200NM      2226.0  Marshall Islands      MHL  Marshall Islands   

   MRGID_TER2  MRGID_SOV2  ... ISO_SOV3 UN_SOV1 UN_SOV2  UN_SOV3  UN_TER1  \
0  

In [21]:
KIR = PNA.loc[PNA['ISO_TER1'] == 'KIR']

In [22]:
KIR_2020 = GFW2020Points.clip(KIR)

In [23]:
print(KIR_2020)

               mmsi        date  cell_ll_lat  cell_ll_lon   hours  \
45458986  440823000  2020-10-02         -4.1        173.6  0.4080   
39661940  416253900  2020-07-29         -4.1        173.6  0.0827   
31374546  412439604  2020-09-19         -4.1        173.5  0.6422   
45547025  440900000  2020-08-17         -4.1        173.5  0.5552   
47968460  554126120  2020-09-15         -4.1        173.6  0.3547   
...             ...         ...          ...          ...     ...   
48022454  572975210  2020-02-18          5.5       -160.5  3.1805   
22942885  367640000  2020-10-06          5.6       -160.5  0.5038   
22942884  367640000  2020-10-06          5.6       -160.6  0.6686   
45754792  441084000  2020-03-13          6.4       -160.3  0.7061   
45754791  441084000  2020-03-13          6.4       -160.4  0.6816   

          fishing_hours                    geometry  
45458986          0.000  POINT (173.60000 -4.10000)  
39661940          0.000  POINT (173.60000 -4.10000)  
31374546 

In [140]:
#PNAEEZs_2020 = GFW2020Points.clip(PNA)

In [141]:
#print(PNAEEZs_2020)

In [24]:
del(GFW2020Points)

### Add flag and gear type

In [25]:
flag_gear_df = pd.read_csv('fishing-vessels-v2.csv')

In [26]:
flag_gear_df.head(2)

,mmsi,flag_ais,flag_registry,flag_gfw,vessel_class_inferred,vessel_class_inferred_score,vessel_class_registry,vessel_class_gfw,self_reported_fishing_vessel,length_m_inferred,...,registries_listed,fishing_hours_2012,fishing_hours_2013,fishing_hours_2014,fishing_hours_2015,fishing_hours_2016,fishing_hours_2017,fishing_hours_2018,fishing_hours_2019,fishing_hours_2020
0,40120000,NaN,GRC,GRC,NaN,NaN,trawlers,trawlers,True,NaN,...,EU|GFCM|IMO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,674.10
1,444002410,NaN,KOR,KOR,NaN,NaN,tuna_purse_seines,tuna_purse_seines,False,NaN,...,FFA|IMO|ISSF|KOR|WCPFC|WCPFC2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144.43


In [27]:
KIR_2020 = pd.merge(KIR_2020, flag_gear_df[['mmsi','flag_gfw','flag_ais','vessel_class_gfw']], on='mmsi', how='left')

In [28]:
KIR_2020.head(2)

,mmsi,date,cell_ll_lat,cell_ll_lon,hours,fishing_hours,geometry,flag_gfw,flag_ais,vessel_class_gfw
0,440823000,2020-10-02,-4.1,173.6,0.4080,0.0,POINT (173.60000 -4.10000),KOR,KOR,tuna_purse_seines
1,416253900,2020-07-29,-4.1,173.6,0.0827,0.0,POINT (173.60000 -4.10000),TWN,TWN,tuna_purse_seines


In [29]:
len(KIR_2020)

353173

In [30]:
# Exporting 2020 effort file for all vessel types in KIR EEZ
KIR_2020.to_csv("KIR_2020.csv")

In [33]:
# Filter for tuna purse seiners
KIR_2020_TunaPurseSeines = KIR_2020[KIR_2020['vessel_class_gfw'] == 'tuna_purse_seines']

In [34]:
print(KIR_2020_TunaPurseSeines.head())

        mmsi        date  cell_ll_lat  cell_ll_lon   hours  fishing_hours  \
0  440823000  2020-10-02         -4.1        173.6  0.4080          0.000   
1  416253900  2020-07-29         -4.1        173.6  0.0827          0.000   
2  412439604  2020-09-19         -4.1        173.5  0.6422          0.145   
4  554126120  2020-09-15         -4.1        173.6  0.3547          0.000   
6  416253900  2020-09-30         -4.1        174.6  0.6497          0.000   

                     geometry flag_gfw flag_ais   vessel_class_gfw  
0  POINT (173.60000 -4.10000)      KOR      KOR  tuna_purse_seines  
1  POINT (173.60000 -4.10000)      TWN      TWN  tuna_purse_seines  
2  POINT (173.50000 -4.10000)      CHN      CHN  tuna_purse_seines  
4  POINT (173.60000 -4.10000)      NRU      NaN  tuna_purse_seines  
6  POINT (174.60000 -4.10000)      TWN      TWN  tuna_purse_seines  


In [35]:
len(KIR_2020_TunaPurseSeines)

180871

### Add imo and other vessel identity attributes

In [36]:
identity_auth_df = pd.read_csv('identity_authorization_v20220701.csv')

In [37]:
identity_core_df = pd.read_csv('identity_core_v20220701.csv')

In [38]:
identity_owner_df = pd.read_csv('identity_owner_v20220701.csv')

In [39]:
print(identity_auth_df.head())

                                    vessel_record_id      ssvid n_shipname  \
0  AUS-191901|CCSBT-FV01286|IMO-8565355|IOTC-8593...  503555500     FISCO1   
1  AUS-191901|CCSBT-FV01286|IMO-8565355|IOTC-8593...  503555500     FISCO1   
2  AUS-191901|CCSBT-FV01286|IMO-8565355|IOTC-8593...  503555500     FISCO1   
3  AUS-191901|CCSBT-FV01286|IMO-8565355|IOTC-8593...  503555500     FISCO1   
4  AUS-191901|CCSBT-FV01286|IMO-8565355|IOTC-8593...  503555500     FISCO1   

  n_callsign        imo flag          authorized_from  \
0    VHN9944  8565355.0  AUS  2012-05-11 00:00:00 UTC   
1    VHN9944  8565355.0  AUS  2015-05-18 00:00:00 UTC   
2    VHN9944  8565355.0  AUS  2016-04-27 00:00:00 UTC   
3    VHN9944  8565355.0  AUS  2018-05-03 00:00:00 UTC   
4    VHN9944  8565355.0  AUS  2019-03-20 00:00:00 UTC   

             authorized_to source_code  
0  2014-11-30 00:00:00 UTC       CCSBT  
1  2015-11-30 00:00:00 UTC       CCSBT  
2  2017-11-30 00:00:00 UTC       CCSBT  
3  2018-11-30 00:00:00 U

In [40]:
print(identity_core_df.head())

                                    vessel_record_id      ssvid  \
0                                         AUS-152391  503010790   
1                             AUS-178452|IMO-5348380  503127520   
2  AUS-191901|CCSBT-FV01286|IMO-8565355|IOTC-8593...  503555500   
3                             AUS-191908|IMO-7319759  503083330   
4                                         AUS-196766  503096150   

          shipname      n_shipname n_callsign        imo flag  \
0    JULIE BURGESS    JULIEBURGESS       VLJV        NaN  AUS   
1           TACOMA          TACOMA        NaN  5348380.0  AUS   
2          FISCO 1          FISCO1    VHN9944  8565355.0  AUS   
3  RUBY ENTERPRISE  RUBYENTERPRISE     VL2084  7319759.0  AUS   
4     OTWAY HUNTER     OTWAYHUNTER        NaN        NaN  AUS   

           first_timestamp           last_timestamp            geartype  \
0  2014-12-08 08:04:56 UTC  2022-06-26 05:24:32 UTC             fishing   
1  2022-02-09 05:51:06 UTC  2022-06-30 23:13:55 UTC     

In [41]:
print(identity_owner_df.head())

                                    vessel_record_id      ssvid  \
0                                         AUS-152391  503010790   
1                             AUS-178452|IMO-5348380  503127520   
2  AUS-191901|CCSBT-FV01286|IMO-8565355|IOTC-8593...  503555500   
3                             AUS-191908|IMO-7319759  503083330   
4                             AUS-191908|IMO-7319759  503083330   

       n_shipname n_callsign        imo flag                    owner  \
0    JULIEBURGESS       VLJV        NaN  AUS                      NaN   
1          TACOMA        NaN  5348380.0  AUS                      NaN   
2          FISCO1    VHN9944  8565355.0  AUS    DDD LAVALLE FISHERIES   
3  RUBYENTERPRISE     VL2084  7319759.0  AUS                      NaN   
4  RUBYENTERPRISE     VL2084  7319759.0  AUS  RUBY MARINE ENGINEERING   

  owner_flag         source_code  
0        AUS                 AUS  
1        AUS                 AUS  
2        AUS  CCSBT, IOTC, WCPFC  
3        AUS      

In [42]:
identity_auth_df.rename(columns={'ssvid': 'mmsi'}, inplace=True)
identity_core_df.rename(columns={'ssvid': 'mmsi'}, inplace=True)
identity_owner_df.rename(columns={'ssvid': 'mmsi'}, inplace=True)

In [43]:
print(identity_auth_df.head())

                                    vessel_record_id       mmsi n_shipname  \
0  AUS-191901|CCSBT-FV01286|IMO-8565355|IOTC-8593...  503555500     FISCO1   
1  AUS-191901|CCSBT-FV01286|IMO-8565355|IOTC-8593...  503555500     FISCO1   
2  AUS-191901|CCSBT-FV01286|IMO-8565355|IOTC-8593...  503555500     FISCO1   
3  AUS-191901|CCSBT-FV01286|IMO-8565355|IOTC-8593...  503555500     FISCO1   
4  AUS-191901|CCSBT-FV01286|IMO-8565355|IOTC-8593...  503555500     FISCO1   

  n_callsign        imo flag          authorized_from  \
0    VHN9944  8565355.0  AUS  2012-05-11 00:00:00 UTC   
1    VHN9944  8565355.0  AUS  2015-05-18 00:00:00 UTC   
2    VHN9944  8565355.0  AUS  2016-04-27 00:00:00 UTC   
3    VHN9944  8565355.0  AUS  2018-05-03 00:00:00 UTC   
4    VHN9944  8565355.0  AUS  2019-03-20 00:00:00 UTC   

             authorized_to source_code  
0  2014-11-30 00:00:00 UTC       CCSBT  
1  2015-11-30 00:00:00 UTC       CCSBT  
2  2017-11-30 00:00:00 UTC       CCSBT  
3  2018-11-30 00:00:00 U

In [44]:
identity_table = pd.merge(identity_core_df, identity_auth_df, on=['vessel_record_id', 'mmsi', 'n_shipname', 'n_callsign', 'imo', 'flag'], how='left', suffixes=('', '_auth'))
                          

In [45]:
identity_table = pd.merge(identity_table, identity_owner_df, on=['vessel_record_id', 'mmsi', 'n_shipname', 'n_callsign', 'imo', 'flag'], how='left', suffixes=('', '_owner'))


In [46]:
print(identity_table.head())

                                    vessel_record_id       mmsi  \
0                                         AUS-152391  503010790   
1                             AUS-178452|IMO-5348380  503127520   
2  AUS-191901|CCSBT-FV01286|IMO-8565355|IOTC-8593...  503555500   
3  AUS-191901|CCSBT-FV01286|IMO-8565355|IOTC-8593...  503555500   
4  AUS-191901|CCSBT-FV01286|IMO-8565355|IOTC-8593...  503555500   

        shipname    n_shipname n_callsign        imo flag  \
0  JULIE BURGESS  JULIEBURGESS       VLJV        NaN  AUS   
1         TACOMA        TACOMA        NaN  5348380.0  AUS   
2        FISCO 1        FISCO1    VHN9944  8565355.0  AUS   
3        FISCO 1        FISCO1    VHN9944  8565355.0  AUS   
4        FISCO 1        FISCO1    VHN9944  8565355.0  AUS   

           first_timestamp           last_timestamp            geartype  ...  \
0  2014-12-08 08:04:56 UTC  2022-06-26 05:24:32 UTC             fishing  ...   
1  2022-02-09 05:51:06 UTC  2022-06-30 23:13:55 UTC             fishin

In [7]:
# Reading in a 2020 effort file with no decimals under 'mmsi'
#KIR_2020 = pd.read_csv('KIR_2020.csv')

In [47]:
merged_df = pd.merge(KIR_2020_TunaPurseSeines,identity_table, on='mmsi', how='left')
                          

In [52]:
print(merged_df.head())

        mmsi        date  cell_ll_lat  cell_ll_lon   hours  fishing_hours  \
0  440823000  2020-10-02         -4.1        173.6  0.4080            0.0   
1  440823000  2020-10-02         -4.1        173.6  0.4080            0.0   
2  440823000  2020-10-02         -4.1        173.6  0.4080            0.0   
3  416253900  2020-07-29         -4.1        173.6  0.0827            0.0   
4  416253900  2020-07-29         -4.1        173.6  0.0827            0.0   

                     geometry flag_gfw flag_ais   vessel_class_gfw  ...  \
0  POINT (173.60000 -4.10000)      KOR      KOR  tuna_purse_seines  ...   
1  POINT (173.60000 -4.10000)      KOR      KOR  tuna_purse_seines  ...   
2  POINT (173.60000 -4.10000)      KOR      KOR  tuna_purse_seines  ...   
3  POINT (173.60000 -4.10000)      TWN      TWN  tuna_purse_seines  ...   
4  POINT (173.60000 -4.10000)      TWN      TWN  tuna_purse_seines  ...   

  is_bunker                                 source_code multi_identity  \
0     False 

In [49]:
len(merged_df)

574885

In [53]:
# Remove duplicate observations
merged_df_EDIT = merged_df.drop_duplicates(subset=['mmsi','date','cell_ll_lat','cell_ll_lon', 'hours', 'fishing_hours'], keep='first')

In [55]:
# Checking that dataset is the correct length
len(merged_df_EDIT)

180871

In [54]:
print(merged_df_EDIT.columns)

Index(['mmsi', 'date', 'cell_ll_lat', 'cell_ll_lon', 'hours', 'fishing_hours',
       'geometry', 'flag_gfw', 'flag_ais', 'vessel_class_gfw',
       'vessel_record_id', 'shipname', 'n_shipname', 'n_callsign', 'imo',
       'flag', 'first_timestamp', 'last_timestamp', 'geartype', 'length_m',
       'tonnage_gt', 'engine_power_kw', 'is_fishing', 'is_carrier',
       'is_bunker', 'source_code', 'multi_identity', 'timestamp_overlap',
       'authorized_from', 'authorized_to', 'source_code_auth', 'owner',
       'owner_flag', 'source_code_owner'],
      dtype='object')


In [56]:
merged_df_EDIT = merged_df_EDIT.drop(columns=['is_fishing','is_carrier','is_bunker','multi_identity','first_timestamp','last_timestamp','timestamp_overlap'])


In [57]:
merged_df_EDIT = merged_df_EDIT.drop(columns=['authorized_from','authorized_to','source_code_auth','source_code','shipname'])

In [64]:
print(merged_df_EDIT.columns)

Index(['mmsi', 'date', 'cell_ll_lat', 'cell_ll_lon', 'hours', 'fishing_hours',
       'geometry', 'flag_gfw', 'flag_ais', 'vessel_class_gfw',
       'vessel_record_id', 'n_shipname', 'n_callsign', 'imo', 'flag',
       'geartype', 'length_m', 'tonnage_gt', 'engine_power_kw', 'owner',
       'owner_flag', 'source_code_owner'],
      dtype='object')


In [61]:
del merged_df

In [62]:
merged_df_EDIT.to_csv("KIR_2020_identity.csv")

In [63]:
merged_df_EDIT.to_file("KIR_2020_identity.shp")

/tmp/ipykernel_359/1829810837.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  merged_df_EDIT.to_file("KIR_2020_identity.shp")


## Extra step - extracting all tuna purse seine effort globally

In [21]:
flag_gear_df = pd.read_csv('fishing-vessels-v2.csv')

In [22]:
GFW_2020 = pd.merge(GFW2020Points, flag_gear_df[['mmsi','flag_gfw','flag_ais','vessel_class_gfw']], on='mmsi', how='left')

In [23]:
GFW_2020_TunaPurseSeines = GFW_2020[GFW_2020['vessel_class_gfw'] == 'tuna_purse_seines']

In [25]:
print(GFW_2020_TunaPurseSeines.head())

     mmsi        date  cell_ll_lat  cell_ll_lon   hours  fishing_hours  \
272   238  2020-01-11          6.8         79.4  0.3038         0.0000   
273   238  2020-01-11          6.8         79.5  0.0738         0.0000   
274   238  2020-01-11          6.9         79.5  1.4372         0.0000   
275   238  2020-01-11          6.9         79.6  1.5183         0.5986   
276   238  2020-01-11          6.9         79.7  1.4813         0.0000   

                     geometry flag_gfw flag_ais   vessel_class_gfw  
272  POINT (79.40000 6.80000)      NaN      NaN  tuna_purse_seines  
273  POINT (79.50000 6.80000)      NaN      NaN  tuna_purse_seines  
274  POINT (79.50000 6.90000)      NaN      NaN  tuna_purse_seines  
275  POINT (79.60000 6.90000)      NaN      NaN  tuna_purse_seines  
276  POINT (79.70000 6.90000)      NaN      NaN  tuna_purse_seines  


In [26]:
GFW_2020_TunaPurseSeines.to_csv("GFW_2020_TunaPurseSeines.csv")

In [28]:
GFW_2020_TunaPurseSeines.to_file("GFW_2020_TunaPurseSeines.shp")

/tmp/ipykernel_429/2159023661.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  GFW_2020_TunaPurseSeines.to_file("GFW_2020_TunaPurseSeines.shp")
